In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
from loguru import logger
from matbench.bench import MatbenchBenchmark
from matbench.constants import CLF_KEY

from gptchem.gpt_regressor import GPTRegressor
from gptchem.tuner import Tuner

logger.enable("gptchem")

In [3]:
import time

import decorator


def retry(howmany, *exception_types, **kwargs):
    timeout = kwargs.get("timeout", 0.0)  # seconds

    @decorator.decorator
    def tryIt(func, *fargs, **fkwargs):
        for _ in range(howmany):
            try:
                return func(*fargs, **fkwargs)
            except exception_types or Exception as e:
                print(e)
                if timeout is not None:
                    time.sleep(timeout)

    return tryIt

In [4]:
mb = MatbenchBenchmark(
    autoload=True,
    subset=[
          "matbench_expt_gap",
        #"matbench_steels",
    ],
)

2023-01-31 00:22:47 INFO     Initialized benchmark 'matbench_v0.1' with 1 tasks: 
['matbench_expt_gap']


In [5]:
@retry(3, timeout=5)
def train_test_fold(task, fold):
    regressor = GPTRegressor(
        task.metadata["target"], Tuner(n_epochs=8, learning_rate_multiplier=0.02, wandb_sync=False)
    )
    train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # train and validate your model
    regressor.fit(train_inputs, train_outputs.values)

    # Get testing data
    test_inputs = task.get_test_data(fold, include_target=False)

    # Predict on the testing data
    # Your output should be a pandas series, numpy array, or python iterable
    # where the array elements are floats or bools
    predictions = regressor.predict(test_inputs)

    # Record your data!
    task.record(fold, predictions)
    return predictions

In [6]:
from collections import defaultdict

In [10]:
predictions = defaultdict(list)

for task in mb.tasks:
    task.load()

    for fold_ind, fold in enumerate(task.folds):
        if task.is_recorded[fold_ind]:
            print(f"Skipping fold {fold_ind} of {task.dataset_name}")
            continue
        pred = train_test_fold(task, fold)
        predictions[task.dataset_name].append(pred)
        train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # print(f"{task.dataset_name}: MAE  {task.scores['mae']['mean']}")

2023-01-31 08:45:57 INFO     Dataset matbench_expt_gap already loaded; not reloading dataset.


Upload progress: 100%|██████████| 421k/421k [00:00<00:00, 829Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230131_084557/train.jsonl: file-b04uhvQekRJtbj22xWLfv85a


2023-01-31 08:46:00.762 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675151160,
  "events": [
    {
      "created_at": 1675151160,
      "level": "info",
      "message": "Created fine-tune: ft-uwfH6Ym2t6tZpl1M0nJZzCqt",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-uwfH6Ym2t6tZpl1M0nJZzCqt",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420861,
      "created_at": 1675151160,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230131_084557/train.jsonl",
      "id": "file-b04uhvQekRJtbj22xWLfv85a",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "

2023-01-31 10:00:22 INFO     Recorded fold matbench_expt_gap-0 successfully.


...

2023-01-31 11:44:50.719 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675161890,
  "events": [
    {
      "created_at": 1675161890,
      "level": "info",
      "message": "Created fine-tune: ft-Pg7oLij2cnq8XpNuNi6Q33ss",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-Pg7oLij2cnq8XpNuNi6Q33ss",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420263,
      "created_at": 1675161890,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230131_114448/train.jsonl",
      "id": "file-1rDCDOkxfyCBM41ioCcCjyNE",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "

2023-01-31 13:04:49 INFO     Recorded fold matbench_expt_gap-2 successfully.


Upload progress: 100%|██████████| 421k/421k [00:00<00:00, 806Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230131_130449/train.jsonl: file-TYjf71XwA4LJNGgSrJCpK38H


2023-01-31 13:04:52.043 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675166691,
  "events": [
    {
      "created_at": 1675166691,
      "level": "info",
      "message": "Created fine-tune: ft-uv7RQGIVJXe6Ho7is8fsaGgY",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-uv7RQGIVJXe6Ho7is8fsaGgY",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420723,
      "created_at": 1675166691,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230131_130449/train.jsonl",
      "id": "file-TYjf71XwA4LJNGgSrJCpK38H",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "

In [9]:
task.scores

ValueError: Cannot score unless all folds are recorded!; folds [0, 1, 2, 3, 4] not recorded!

In [ ]:
mb.to_file("gpt_expt_gap_bench.json.gz")

2023-01-30 18:53:04 INFO     Successfully wrote MatbenchBenchmark to file 'gpt_steel_bench.json.gz'.
